## 10. Bash Scripting Finale: Auto Web Setup

We are putting into practice what we've learned about bash scripting and what we do here is going to be the base of many other automation tools.

### Setup

* Bring up the VMs: scriptbox, web01, web02 and web03

* Log in to the *scriptbox* VM and the root user

* Go to `/opt/scripts/` and create a directory: `mkdir remote_websetup`.

* In the new directory, create a host file: `vim remhosts` and add the IP addresses or hostnames of the VMs you are connecting (**targets**):

    ```
      1 web01
      2 web02
      3 web03
    ```

* Use a `for` loop to test the login to these machines. Use the `;` to separate lines in the same command line:

    ```
      # for printing:
      for host in `cat remhosts`; do echo $host; done

      # for test the login:
      for host in `cat remhosts`; do ssh devops@$host hostname; done
      for host in `cat remhosts`; do ssh devops@$host uptime; done
    ```

* Copy/Create the script for websetup using variables (`../3_vars_websetup.sh`) to the `remote_websetup` directory. The script contains the following:

    ```
        1 #!/bin/bash
        2
        3 # Variables
        4 PACKAGE="httpd wget unzip"
        5 SVC="httpd"
        6 URL='https://www.tooplate.com/zip-templates/2098_health.zip'
        7 ART_NAME='2098_health'
        8 TEMPDIR="/tmp/webfiles"
        9
      10 # Installing dependencies (> to send output to null - do not show)
      11 echo "#####################################"
      12 echo "Installing packages"
      13 echo "#####################################"
      14 sudo yum install $PACKAGE -y > /dev/null
      15 echo
      16
      17 # Start and enable services
      18 echo "#####################################"
      19 echo "Start & enable HTTPD Service"
      20 echo "#####################################"
      21 sudo systemctl start $SVC
      22 sudo systemctl enable $SVC
      23 echo
      24
      25 # Creating a temporary directory
      26 echo "#####################################"
      27 echo "Starting artifact deployment"
      28 echo "#####################################"
      29 mkdir -p $TEMPDIR
      30 cd $TEMPDIR
      31 echo
      32
      33 # Get and copy webpage template to the /var/www/html directory
      34 wget $URL > /dev/null
      35 unzip $ART_NAME.zip > /dev/null
      36 sudo cp -r $ART_NAME/* /var/www/html/
      37 echo
      38
      39 # Bounce service
      40 echo "#####################################"
      41 echo "Restarting HTTPD Service"
      42 echo "#####################################"
      43 systemctl restart $SVC
      44 echo
      45
      46 # Clean up temporary files
      47 echo "#####################################"
      48 echo "Removing temporary files"
      49 echo "#####################################"
      50 rm -rf $TEMPDIR
      51
      52 # Print status of the service
      53 sudo systemctl status $SVC
      54
      55 # Show the content of the /var/www/html directory
      56 ls /var/www/html/
    ```

* Rename the file: `mv 3_vars_websetup.sh multios_websetup.sh`.

* Modify the file to make it multiOS (currently is only for CentOS): for CentOS and Ubuntu:

    - We need to add some verifications and `if` conditions

    - Comment (`#`) the variable declaration for `PACKAGE` and `SVC` (lines 4 and 5)

    - Before line 10 of the script, verify whether `yum` is part of the VM (CentOS system) or not (Ubuntu system). The code will run according to this condition, i.e., all CentOS-related packages and software are going to be installed if the condition is True. If False, then all packages and software related to Ubuntu should be included:

    ```

    ```